In [1]:
from math import pi
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d, LinearAxis, HoverTool
from bokeh.layouts import column
from datetime import datetime, timedelta
import sys
import backtest_ema2
import mysql_prices as ms

In [ ]:
def recent_gdax(sd_str):
    engine = ms.connect_mysql()
    gdax_df = ms.get_gdax_prices(sd_str)
    return(gdax_df)



def df_resolution(trimmed_df, res_type):
    if res_type == 'day':
        df = trimmed_df.groupby(trimmed_df['time'].dt.date).agg(
    {'open':'first',  'high': max, 'low': min, 'close':'last',
     'volumeto': sum, 'volumefrom': sum,
    }).reset_index()
    elif res_type == 'minute':
        df = trimmed_df
    return(df)

In [21]:
def graph_candlestick(gdf, df, fills, results):
    buys = fills[fills['side']=='buy']
    sells = fills[fills['side']=='sell']
    inc = df.close > df.open
    dec = df.open > df.close
    #w = 18*60*60*1000 # candle period in ms
    res_len_secs = (df.time[2] - df.time[1]).seconds
    w = res_len_secs*1000 # candle period in ms
    hover = HoverTool(tooltips = [
           ("(x,y)", "($x, $y)"),
        ],
        formatters={
            'time': 'datetime', # use 'datetime' formatter for 'date' field
        },
    )
    TOOLS = "crosshair,wheel_zoom,pan,box_zoom,reset,save"
    if results is not None:
        my_title = f"Final Balance: {round(results['usd_bal'],2)}, ROI: {round(results['roi'],2)}, Hodl ROI: {round(results['hodl_roi'],2)}"
    else:
        my_title = "BTC Minute Data"
    p = figure(x_axis_type="datetime", tools=[hover,TOOLS], active_scroll="wheel_zoom", plot_width=800, title = my_title,  y_range=(df['close'].min()*.9, df['close'].max() * 1.1), logo=None)
    p.grid.grid_line_alpha=0.3
    band_x = np.append(gdf.time, gdf.time[::-1])
    band_y = np.append(gdf.lower_ema, gdf.upper_ema[::-1])
    p.patch(band_x, band_y, color='purple', fill_alpha=0.03)
    p.segment(df.time, df.high, df.time, df.low, color="black", alpha = 0.5)
    p.vbar(df.time[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="green", alpha = 0.5)
    p.vbar(df.time[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="red", alpha = 0.5)
    #p.multi_line([list(df.time),list(df.time)],[list(df.r30),list(df.r3)], line_color=['blue', 'orange'])
    p.line(gdf.time, gdf.upper_ema, color="purple", alpha = 0.02)
    p.line(gdf.time, gdf.lower_ema, color="purple", alpha = 0.02)
    #p.line(gdf.time, gdf.r3, color="red", alpha = 0.3)
    #if gdf is not None:
    #    p.line(gdf.time, gdf.r30, color="purple", alpha = 0.3)
    #p.extra_y_ranges = {"foo": Range1d(start=0, end=df.n.max())}
    #p.line(df.time, df.n, color="blue", y_range_name="foo", alpha = 0.4)
    #p.add_layout(LinearAxis(y_range_name="foo"), 'left')
    p.circle(buys['time'], buys['price'], color="green", size=8, alpha=0.5)
    p.circle(sells['time'], sells['price'], color="red", size=8, alpha=0.5)
    return(p)

In [24]:
start_time = "2018-05-01"
end_time = "2018-05-02"

bt_vars = backtest_ema2.BacktestSettings()

factor_high = 1 + .011
factor_low = 1 - .005

bt_vars.set_upper_window(200)
bt_vars.set_lower_window(50)
bt_vars.set_factor_high(factor_high)
bt_vars.set_factor_low(factor_low)
bt_vars.set_sell_pct_btc(.02)
bt_vars.set_buy_amt_usd(250)
bt_vars.set_min_usd(500)
bt_vars.set_min_btc(.001)
bt_vars.set_principle_usd(25000)
#bt_vars.set_principle_btc()

#principle_usd = 25000
#principle_btc = 0

#upper_window = 200
#lower_window = 50

#factor_high = 0.011
#factor_low = 0.005

# Sell percent of bitcoin
#sell_pct_btc = 0.02
#buy_amt_usd = 1000

#min_usd = 250
#min_btc = .001

#raw_data = pd.read_csv('~/coinbase_data.csv')

In [25]:
raw_data = recent_gdax(start_time)
raw_data.head()

,time,open,close,high,low,volume,mean
0,2018-05-01 00:01:00,9239.33,9233.01,9239.34,9233.00,3.837170,9236.1700
1,2018-05-01 00:02:00,9233.00,9230.34,9233.00,9230.34,5.276955,9231.6700
2,2018-05-01 00:03:00,9230.34,9230.34,9230.35,9230.34,1.967550,9230.3425
3,2018-05-01 00:04:00,9226.00,9213.13,9226.01,9213.13,15.680743,9219.5675
4,2018-05-01 00:05:00,9216.16,9213.14,9216.16,9213.13,1.324538,9214.6475


In [26]:
day_df, trimmed_df = backtest_ema2.prep_data(raw_data, start_time, end_time)

In [27]:
trimmed_df, my_res, my_fills = backtest_ema2.single_backtest(
    trimmed_df, bt_vars)




In [28]:
trimmed_df.sell_signal.sum()

0

In [29]:
#fills = get_back_fills_since(start_time)
p = graph_candlestick(trimmed_df, trimmed_df, my_fills, my_res)
output_notebook()
show(p)



Loading BokehJS ...